In [18]:
import pandas as pd
import os
import shutil
import json
from sklearn.model_selection import train_test_split

scatter = []
input_path = '/home/ubuntu/synth-doc-AI/test_run'
dest_path = '/home/ubuntu/synthetic_us_passports'

if not os.path.exists(dest_path):
    os.mkdir(dest_path)
    os.mkdir(os.path.join(dest_path, 'train'))
    os.mkdir(os.path.join(dest_path, 'test'))


meta_files = [file for file in os.listdir(input_path) if file.endswith('.json') and file != 'metadata.json']
train, test = train_test_split(meta_files, test_size=160)

for meta_file in meta_files:
    img_name = meta_file.replace('_metadata.json', '.png')
    split = 'train' if meta_file in train else 'test'
    shutil.copyfile(
        f'{input_path}/{img_name}', 
        f'{dest_path}/{split}/{img_name}'
    )
    meta = json.load(open(os.path.join(input_path, meta_file)))
    scatter.append({'file_name': f'{split}/{img_name}', **meta})

In [19]:
df = pd.DataFrame.from_dict(scatter)

In [33]:
df.head()

,file_name,Type,Passport_Number,Surname,Given_Names,Nationality,DOB,Place_of_Birth,Sex,Date_of_Issue,Authority,Date_of_Expiration,Endorsements
0,train/sample_8982.png,L,899894602,Sharp,Shelby Mcmillan,Gibraltar,23 Dec 2013,Heard Island and McDonald Islands,F,17 May 2011,Lesotho,19 Jan 1995,Laugh wrong look fire to four short.
1,train/sample_1329.png,J,729681650,Acevedo,Frances Rose,Haiti,14 Nov 1981,Guernsey,F,13 Apr 1970,Finland,06 Dec 1982,Amount may threat office.
2,train/sample_8716.png,i,945333004,Zamora,Aaron Randolph,Belgium,30 Nov 1979,Holy See (Vatican City State),F,03 Nov 1998,Falkland Islands (Malvinas),18 Aug 1997,NaN
3,train/sample_5141.png,e,827029691,Ramirez,Jeremy Choi,San Marino,19 Sep 1978,Dominican Republic,F,12 Mar 1997,United States Virgin Islands,20 Jan 1978,NaN
4,train/sample_2714.png,R,909693682,Sims,Courtney Holder,Dominica,25 Oct 1978,Uzbekistan,M,20 Oct 1976,Gibraltar,07 May 1974,Some relationship paper animal natural adminis...


In [21]:
df = df.drop(['Code', 'MRZ_line_1', 'MRZ_line_2'], axis=1)
df = df.rename({'DOB': 'Date_of_Birth'})

In [34]:
df.to_csv(f'{dest_path}/metadata.csv')

In [35]:
from datasets import load_dataset

dataset = load_dataset(dest_path)

Generating train split: 9840 examples [00:01, 7471.40 examples/s]
Generating test split: 160 examples [00:00, 900.16 examples/s]


In [41]:
dataset = dataset.remove_columns('Unnamed: 0')

In [42]:
dataset['train'][0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1783x1859>,
 'Type': 'J',
 'Passport_Number': 604876475,
 'Surname': 'Lamb',
 'Given_Names': 'Kimberly Tyler',
 'Nationality': 'Norfolk Island',
 'DOB': '19 Jan 2009',
 'Place_of_Birth': 'Niue',
 'Sex': 'M',
 'Date_of_Issue': '28 Jul 1986',
 'Authority': 'Canada',
 'Date_of_Expiration': '01 Oct 2021',
 'Endorsements': 'Score sort write nor Mrs beyond.'}

In [44]:
dataset.push_to_hub('arnaudstiegler/v2_synthetic_us_passports_easy')

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 246/246 [00:01<00:00, 163.29 examples/s]

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 246/246 [00:01<00:00, 151.05 examples/s]

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 246/246 [00:01<00:00, 171.81 examples/s]

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 246/246 [00:01<00:

CommitInfo(commit_url='https://huggingface.co/datasets/arnaudstiegler/v2_synthetic_us_passports_easy/commit/b00fb6ca7550330a3f7d37d91d08cb2ac8e8beae', commit_message='Upload dataset', commit_description='', oid='b00fb6ca7550330a3f7d37d91d08cb2ac8e8beae', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
df.loc[df['file_name'] == 'train/sample_0.png']

,file_name,Type,Passport_Number,Surname,Given_Names,Nationality,DOB,Place_of_Birth,Sex,Date_of_Issue,Authority,Date_of_Expiration,Endorsements
5210,train/sample_0.png,J,604876475,Lamb,Kimberly Tyler,Norfolk Island,19 Jan 2009,Niue,M,28 Jul 1986,Canada,01 Oct 2021,Score sort write nor Mrs beyond.


In [30]:
df['DOB'].unique().size

4358